In [3]:
'''
2022.10.6，复现FinEvent Model
paper: from Reinforced, Incremental and Cross-lingual Event Detection From Social Messages
github address: https://github.com/RingBDStack/FinEvent
'''
'''
settings:
    matplotlib==3.5.1
    networkx==2.6.3
    numpy==1.22.0
    pandas==1.3.5
    scikit-learn==1.0.1
    scipy==1.7.3
    torch==1.10.0
    torch-cluster==1.5.9
    torch-geometric==2.0.2
    torch-scatter==2.0.9
    torch-sparse==0.6.12
    torch-spline-conv==1.2.1
    tqdm==4.62.3
    dgl==0.4.3
'''

'\nsettings:\n    matplotlib==3.5.1\n    networkx==2.6.3\n    numpy==1.22.0\n    pandas==1.3.5\n    scikit-learn==1.0.1\n    scipy==1.7.3\n    torch==1.10.0\n    torch-cluster==1.5.9\n    torch-geometric==2.0.2\n    torch-scatter==2.0.9\n    torch-sparse==0.6.12\n    torch-spline-conv==1.2.1\n    tqdm==4.62.3\n    dgl==0.4.3\n'

In [7]:
import numpy as np
import pandas as pd
from datetime import datetime

import networkx as nx
from scipy import sparse

import torch

import os
project_path = os.getcwd()

In [8]:
project_path

'D:\\PycharmProjects\\GNN_Event_Detection_models\\FinEvent Models'

# layers

## layer

In [48]:
from torch.functional import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch.nn import Linear, BatchNorm1d, Sequential, ModuleList, ReLU, Dropout

In [49]:
class GAT(nn.Module):
    '''
    adopt this module when using mini-batch
    '''
    def __init__(self, in_dim, hid_dim, out_dim, heads) -> None:
        super(GAT, self).__init__()
        self.GAT1 = GATConv(in_channels=in_dim, out_channels=hid_dim, heads=heads, add_self_loops=False)
        self.GAT2 = GATConv(in_channels=hid_dim*heads, out_channels=out_dim, add_self_loops=False)
        self.layers = ModuleList([self.GAT1, self.GAT2])
        self.norm = BatchNorm1d(heads * hid_dim)
    
    def forward(self, x, adjs, device):
        for i, (edge_index, _, size) in enumerate(adjs):
            # x: Tensor, edge_index: Tensor
            x, edge_index = x.to(device), edge_index.to(device)
            x_target = x[:size[1]]  # Target nodes are always placed first
            x = self.layers[i]((x,x_target), edge_index)
            if i == 0:
                x = self.norm(x)  # 归一化操作，防止梯度散射
                x = F.elu(x)  # 非线性激活函数elu
                x = F.dropout(x, training=self.training)
            del edge_index
        return edge_index

In [50]:
class Intra_AGG(nn.Module):  # intra-aggregation
    def __init__(self, GAT_args):
        super(Intra_AGG, self).__init__()
        in_dim, hid_dim, out_dim, heads = GAT_args
        self.gnn = GAT(in_dim, hid_dim, out_dim, heads)
    
    def forward(self, x, adjs, device):
        x = self.gnn(x, adjs, device)
        return x
    

In [51]:
class Inter_AGG(nn.Module):  # inter-aggregation
    def __init__(self, mlp_args=None):
        super(Inter_AGG, self).__init__()
        if mlp_args is not None:
            hid_dim, out_dim = mlp_args
            self.mlp = nn.Sequential(
                        Linear(hid_dim, hid_dim),
                        BatchNorm1d(hid_dim),
                        ReLU(inplace=True),
                        Dropout(),
                        Linear(hid_dim, out_dim),
                        )
    def forward(self, features, thresholds, inter_opt):
        batch_size = features[0].size(0)
        features = torch.transpose(features, dim0=0, dim1=1)
        if inter_opt == 'cat_wo_avg':
            features = features.reshape(batch_size, -1)
        elif inter_opt == 'cat_w_avg':
            # weighted average and concatenate
            features = torch.mul(features, thresholds).reshape(batch_size, -1)
        elif inter_opt == 'cat_w_avg_mlp':
            features = torch.mul(features, thresholds).reshape(batch_size, -1)
            features = self.mlp(features)
        elif inter_opt == 'cat_wo_avg_mlp':
            features = torch.mul(features, thresholds).reshape(batch_size, -1)
            features = self.mlp(features)
        elif inter_opt == 'add_wo_avg':
            features = features.sum(dim=1)
        elif inter_opt == 'add_w_avg':
            features = torch.mul(features, thresholds).sum(dim=1)
        return features
            

## TripletLoss

In [56]:
from itertools import combinations
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc

In [57]:
# Applies an average on seq, of shape(nodes, features)
class AvgReadout(nn.Module):
    def __init__(self):
        super(AvgReadout, self).__init__()
    def forward(self, seq):
        return torch.mean(seq, 0)

In [60]:
class Discriminator(nn.Module):  # 鉴别器
    def __init__(self, n_h):
        super(Discriminator, self).__init__()
        self.f_k = nn.Bilinear(n_h, n_h, 1)  # 双向现行变换x1*A*x2
        for m in self.modules():
            self.weights_init(m)
    
    def weights_init(self, m):
        if isinstance(m, m.Bilinear):
            torch.nn.init.xavier_uniform_(m.weight.data)  # 权值初始化方法，均分分布
            if m.bias is not None:
                m.bias.data.fill_(0.0)
    
    def forward(self, c, h_pl, h_mi, s_bias1=None, s_bias2=None):
        c_x = torch.unsqueeze(c, 0)
        c_x = c_x.expand_as(h_pl)  # torch.randn(size*)生成size维数组；expand是扩展到size_new数组；expand_as是扩展到像y的数组
        sc_1 = torch.squeeze(self.f_k(h_pl, c_x), 1)
        sc_2 = torch.squeeze(self.f_k(h_mi, c_x), 1)
        if s_bias1 is not None:
            sc_1 += s_bias1
        if s_bias2 is not None:
            sc_2 += s_bias2
        logits = torch.cat((sc_1, sc_2), 0)
        return logits

In [61]:
class OnlineTripletLoss(nn.Module):
    '''
    Online Triplets loss
    Takes a batch of embeddings and corresponding labels
    Triplets are generated using triplet_selector objects that take embeddings and targets and return indices of triplets
    '''
    def __init__(self, margin, triplet_selector):
        super(OnlineTripletLoss, self).__init__()
        self.margin = margin
        self.triplet_selector = triplet_selector
    
    def forward(self, embeddings, target):
        triplets = self.triplet_selector.get_triplets(embeddings, target)
        # if embeddings.is_cuda():
        #     triplets = triplets.cuda()
        ap_distances = (embeddings[triplets[:,0]] - embeddings[triplets[:,1]]).pow(2).sum(1) # .pow(.5)
        an_distances = (embeddings[triplets[:,0]] - embeddings[triplets[:,2]]).pow(2).sum(1) # .pow(.5)
        losses = F.relu(ap_distances - an_distances + self.margin)
        
        return losses.mean(), len(triplets)

In [63]:
def pdist(vectors):
    distance_matrix=-2*vectors.mm(torch.t(vectors))+vectors.pow(2).sum(dim=1).view(1,-1)+vectors.pow(2).sum(dim=1).view(1,-1)
    return distance_matrix

In [64]:
class TripletSelector:
    '''
    Implementation should return indices of anchors, positive and negative samples
    return np array of shape [N_triplets * 3]
    '''
    def __init__(self):
        pass
    def get_triplets(self, embeddings, labels):
        raise NotImplementedError  # 如果这个方法没有被子类重写，但是调用了，就会报错。

In [65]:
class FunctionNegativeTripletSelector(TripletSelector):
    '''
    For each positive pair, takes the hardes negative sample (with the greatest triplet loss value) to create a triplet
    Margin should match the margin userd in triplet loss.
    negative_selection_fn should take array of loss_values for a given anchor-positive pair and all negative samples
    and return a negative index for that pair
    '''
    def __init__(self, margin, negative_selection_fn, cpu=True):
        super(FunctionNegativeTripletSelector, self).__init__()
        self.cpu = cpu
        self.margin = margin
        self.negative_selection_fn = negative_selection_fn
    
    def get_triplets(self, embeddings, labels):
        if self.cpu:
            embeddings = embeddings.cpu()
        distance_matrix = pdist(embeddings)
        distance_matrix = distance_matrix.cpu()
        
        labels = labels.cpu().data.numpy()
        triplets = []
        
        for label in set(labels):
            label_mask = (labels == label)
            label_indices = np.where(label_mask)[0]
            if len(label_indices) < 2:
                continue
            negative_indices = np.where(np.logical_not(label_mask))[0]
            anchor_positives = list(combinations(label_indices, 2)) # all anchor-positive pairs
            anchor_positives = np.array(anchor_positives)
            
            ap_distances = distance_matrix[anchor_positives[:,0], anchor_positives[:,1]]
            for anchor_positive, ap_distance in zip(anchor_positives, ap_distances):
                loss_values = ap_distance - distance_matrix[
                    torch.LongTensor(np.array([anchor_positive[0]])), torch.LongTensor(negative_indices)] + self.margin
                loss_values = loss_values.data.cpu().numpy()
                hard_negative = self.negative_selection_fn(loss_values)
                if hard_negative is not None:
                    hard_negative = negative_indices[hard_negative]
                    triplets.append([anchor_positive[0], anchor_positive[1], hard_negative])
        
        if len(triplets) == 0:
            triplets.append([anchor_positive[0], anchor_positive[1], negative_indices[0]])
        
        triplets = np.array(triplets)
        return torch.LongTensor(triplets)

In [66]:
def random_hard_negative(loss_values):
    hard_negatives = np.where(loss > 0)[0]
    return np.random.choice(hard_negatives) if len(hard_negatives) > 0 else None

In [67]:
def hardest_negative(loss_values):
    hard_negative = np.argmax(loss_values)
    return hard_negative if loss_values[hard_negative] > 0 else None

In [68]:
def HardestNegativeTripletSelector(margin, cpu=False):
    return FunctionNegativeTripletSelector(margin=margin, negative_selection_fn=hardest_negative, cpu=cpu)

In [69]:
def RandomNegativeTripletSelector(margin, cpu=False):
    return FunctionNegativeTripletSelector(margin=margin, negative_selection_fn=random_hard_negative, cpu=cpu)

In [58]:
torch.randn(2, 1, 1)

tensor([[[-0.9853]],

        [[-0.6914]]])

In [59]:
torch.randn(2, 3, 3)

tensor([[[-0.9285, -0.4395,  1.1670],
         [-1.4341, -0.8047,  0.6164],
         [ 0.4023, -0.7834, -1.9064]],

        [[ 1.3818,  0.0470,  1.3903],
         [ 1.1434,  0.7092, -0.8992],
         [-0.5288, -0.4024, -1.4602]]])

## neighborRL

In [71]:
from typing import Any, Dict
import numpy as np
import torch
from torch.functional import Tensor
import math
import os

In [72]:
def pre_node_dist(multi_r_data, features, save_path=None):
    '''
    This is used to culculate the similarity between node and 
    its neighbors in advance in order to avoid the repetitive computation.
    Args:
        multi_r_data ([type]): [description]
        features ([type]): [description]
        save_path ([type], optional): [description]. Defaults to None.
    '''
    relation_config: Dict[str, Dict[int, Any]] = {}
    for relation_id, r_data in enumerate(multi_r_data):
        node_config: Dict[int, Any] = {}
        r_data: Tensor
        unique_nodes = r_data[1].unique()
        num_nodes = unique_nodes.size(0)
        for node in range(num_nodes):
            # get neighbors' index
            neighbors_idx = torch.where(r_data[1]==node)[0]
            # get neghbors
            neighbors = r_data[0, neighbors_idx]
            num_neighbors = neighbors.size(0)
            neighbors_features = features[neighbors, :]
            target_features = features[node, :]
            # calculate enclidean distance with broadcast
            dist: Tensor = torch.norm(neighbors_features - target_features, p=2, dim=1)  # torch.norm求a列维度(dim指定)的2(p指定)范数(长度)
            # smaller is better and we use 'top p' in our paper
            # (threshold * num_neighbors) see RL_neighbor_filter for details
            sorted_neighbors, sorted_index = dist.sort(descending=False)
            node_config[node] = {'neighbors_idx': neighbors_idx,
                                'sorted_neighbors': sorted_neighbors,
                                'sorted_index': sorted_index,
                                'num_neighbors': num_neighbors}
        relation_config['relation_%d' % relation_id] = node_config
    if save_path is not None:
        save_path = os.path.join(save_path, 'relation_config.npy')
        np.save(save_path, relation_config)

In [73]:
def RL_neighbor_filter(multi_r_data, RL_thtesholds, load_path):
    load_path = os.path.join(load_path, 'relation_config.npy')
    relation_config = np.load(load_path, allow_pickle=True)
    relation_config = relation_config.tolist()
    relations = list(relation_config.keys())
    multi_remain_data = []
    
    for i in range(len(relations)):
        edge_index: Tensor = multi_r_data[i]
        unique_nodes = edge_index[1].unique()
        num_nodes = unique_nodes.size(0)
        remain_node_index = torch.tensor([])
        for node in range(num_nodes):
            # extract config
            neighbors_idx = relation_config[relations[i]][node]['neighbors_idx']
            num_neighbors = relation_config[relations[i]][node]['num_neighbors']
            sorted_neighbors = relation_config[relations[i]][node]['sorted_neighbors']
            sorted_index = relation_config[relation[i]][node]['sorted_index']
            
            if num_neighbors < 5:
                remain_node_index = torch.cat((remain_node_index, neighbors_idx))
                continue  # add limitations
            
            threshold = float(RL_thtesholds[i])
            
            num_kept_neighbors_idx = neighbors_idx[sorted_index[:num_kept_neighbors]]
            filtered_neighbors_idx = neighbors_idx[sorted_index[:num_kept_neighbors]]
            remain_node_index = torch.cat((remain_node_index, filtered_neighbors_idx))
        remain_node_index = remain_node_index.type('torch.LongTensor')
        edge_index = edge_index[:, remain_node_index]
        multi_remain_data.append(edge_index)
    
    return multi_remain_data

# MarGNN

In [52]:
import torch
import torch.nn as nn
from torch.functional import Tensor
import time

In [54]:
class MarGNN(nn.Module):
    def __init__(self, GNN_args, num_relations, inter_opt, is_shared=False):
        super(MarGNN, self).__init__()
        
        self.num_relations = num_relations
        self.inter_opt = inter_opt
        self.is_shared = is_shared
        if not self.is_shared:
            self.intra_aggs = torch.nn.ModuleList([Intra_AGG(GNN_args) for _ in range(self.num_relations)])
        else:
            self.intra_aggs = Intra_AGG(GNN_args) # shared parameters
        
        if self.inter_opt == 'cat_w_avg_mlp' or 'cat_wo_avg_mlp':
            in_dim, hid_dim, out_dim, heads = GNN_args
            mlp_args = self.num_relations * out_dim, out_dim
        else:
            self.inter_agg = Inter_AGG()
    
    def forward(self, x, adjs, n_ids, device, RL_thresholds):
        # RL_threshold: tensor([[.5], [.5], [.5]])
        if RL_thresholds is None:
            RL_thresholds = torch.FloatTensor([[1.], [1.], [1.]])
        if not isinstance(RL_thresholds, Tensor):
            RL_thresholds = torch.FloatTensor(RL_thresholds)
        RL_thresholds = RL_thresholds.to(device)
        
        features = []
        for i in range(self.num_relations):
            if not self.is_shared:
                # print('Intra Aggregation of relation %d' % i)
                features.append(self.intra_aggs[i](x[n_ids[i]], adjs[i], device))
            else:
                # shared parameters
                # print('Shared Intra Aggregation ...')
                features.append(self.intra_aggs(x[n_ids[i]], adjs[i], device))
        
        features = torch.stack(features, dim=0)
        features = self.inter_agg(features, RL_thresholds, self.inter_opt)
        
        return features

# utils

## step 1: generate initial features

In [9]:
# generate the initial features for the messages
'''
This file generates the initial message features (please see Figure 1(b) and Section 3.2 of the paper for more details).
To leverage the semantics in the data, we generate document feature for each message,
which is calculated as an average of the pre-trained word embeddings of all the words in the message
We use the word embeddings pre-trained by en_core_web_lg, while other options, 
such as word embeddings pre-trained by BERT, are also applicable.
To leverage the temporal information in the data, we generate temporal feature for each message,
which is calculated by encoding the times-tamps: we convert each timestamp to OLE date, 
whose fractional and integral components form a 2-d vector.
The initial feature of a message is the concatenation of its document feature and temporal feature.
'''

'\nThis file generates the initial message features (please see Figure 1(b) and Section 3.2 of the paper for more details).\nTo leverage the semantics in the data, we generate document feature for each message,\nwhich is calculated as an average of the pre-trained word embeddings of all the words in the message\nWe use the word embeddings pre-trained by en_core_web_lg, while other options, \nsuch as word embeddings pre-trained by BERT, are also applicable.\nTo leverage the temporal information in the data, we generate temporal feature for each message,\nwhich is calculated by encoding the times-tamps: we convert each timestamp to OLE date, \nwhose fractional and integral components form a 2-d vector.\nThe initial feature of a message is the concatenation of its document feature and temporal feature.\n'

In [10]:
import en_core_web_lg

In [11]:
load_path = '../datasets/Twitter/'
save_path = '../datasets/Twitter/'

In [12]:
# load dataset
p_part1 = load_path + '68841_tweets_multiclasses_filtered_0722_part1.npy'
p_part2 = load_path + '68841_tweets_multiclasses_filtered_0722_part2.npy'
df_np_part1 = np.load(p_part1, allow_pickle=True)  # allow_pickle, Allow loading pickled object arrays stored in npy files
df_np_part2 = np.load(p_part2, allow_pickle=True)
df_np = np.concatenate((df_np_part1, df_np_part2),axis=0)
print('loaded data.')
df = pd.DataFrame(data=df_np, columns=['event_id','tweet_id','text','user_id','created_at','user_loc','place_type',
                                      'place_full_name','place_country_code','hashtags','user_mentions','image_urls',
                                      'entities','words','filtered_words','sampled_words'])
print('Data converted to dataframe.')

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/Twitter/68841_tweets_multiclasses_filtered_0722_part1.npy'

In [ ]:
print(df.shape)
print(df.head(10))

In [ ]:
# calculate the embeddings of all the documents in the dataframe
# the embeddings of each document is an average of the pre-trained embeddings of all the words in it
def documents_to_features(df):
    nlp = en_core_web_lg.load()
    features = df.filtered_words.apply(lambda x: nlp(' '.join(x)).vector).values
    return np.stack(features, axis=0)

In [ ]:
# encode one times-tamp
# t_str: a string of format '2012-10-11 07:19:34'
def extract_time_feature(t_str):
    t = datetime.fromisoformat(str(t_str)) # 分别返回年月日时分秒列表
    OLE_TIME_ZERO = datetime(1899, 12, 30)
    delta = t - OLE_TIME_ZERO
    return [(float(delta.days)/10000.), (float(delta.seconds)/86400)] # 86400 seconds in day

In [ ]:
# encode the times-tamps of all the messages in the dateframe
def df_to_t_features(df):
    t_features = np.asarray([extract_time_feature(t_str) for t_str in df['created_at']])
    return t_features

In [ ]:
d_features = documents_to_features(df)
print('Document features generated')

In [ ]:
t_features = df_to_t_features(df)
print('Time features generated.')

In [ ]:
combined_features = np.concatenate((d_features, t_features),axis=1)
print('Concatenated document features and time features.')

In [ ]:
np.save(save_path + 'features_69612_0709_spacy_lg_zero_multiclasses_filtered.npy', combined_featurs)
print('Initial features saved.')

In [ ]:
combined_features = np.load(save_path + 'features_69612_0709_spacy_lg_zero_multiclasses_filtered.npy')
print('Initial features loaded.')
print(combined_features.shape)

In [13]:
datetime(1899, 12, 30)

datetime.datetime(1899, 12, 30, 0, 0)

In [14]:
datetime.fromisoformat('2012-10-11 07:19:34')

datetime.datetime(2012, 10, 11, 7, 19, 34)

## step 2: custom message graph

In [15]:
# construct incremental message graphs

## construct_graph

In [16]:
'''
KPGNN:
    This file splits the Twitter dataset into 21 message blocks (please see Section 4.3 of the paper for more details), 
    use the message blocks to construct heterogeneous social graphs (please see Figure 1(a) and Section 3.2 of the paper for more details) 
    and maps them into homogeneous message graphs (Figure 1(c)).
    Note that:
    # 1) We adopt the Latest Message Strategy (which is the most efficient and gives the strongest performance. See Section 4.4 of the paper for more details) here, 
    # as a consequence, each message graph only contains the messages of the date and all previous messages are removed from the graph;
    # To switch to the All Message Strategy or the Relevant Message Strategy, replace 'G = construct_graph_from_df(incr_df)' with 'G = construct_graph_from_df(incr_df, G)' inside construct_incremental_dataset_0922().
    # 2) For test purpose, when calling construct_incremental_dataset_0922(), set test=True, and the message blocks, as well as the resulted message graphs each will contain 100 messages.
    # To use all the messages, set test=False, and the number of messages in the message blocks will follow Table. 4 of the paper.
'''

"\nKPGNN:\n    This file splits the Twitter dataset into 21 message blocks (please see Section 4.3 of the paper for more details), \n    use the message blocks to construct heterogeneous social graphs (please see Figure 1(a) and Section 3.2 of the paper for more details) \n    and maps them into homogeneous message graphs (Figure 1(c)).\n    Note that:\n    # 1) We adopt the Latest Message Strategy (which is the most efficient and gives the strongest performance. See Section 4.4 of the paper for more details) here, \n    # as a consequence, each message graph only contains the messages of the date and all previous messages are removed from the graph;\n    # To switch to the All Message Strategy or the Relevant Message Strategy, replace 'G = construct_graph_from_df(incr_df)' with 'G = construct_graph_from_df(incr_df, G)' inside construct_incremental_dataset_0922().\n    # 2) For test purpose, when calling construct_incremental_dataset_0922(), set test=True, and the message blocks, as we

In [17]:
from time import time
import dgl

Using backend: pytorch


In [18]:
# construct a heterogeneous graph using tweet ids, user_ids, entities and rare(sampled) words(4 modalities模态)
# if G is not None then insert new nodes to G
def construct_graph_from_df(df, G=None):
    if G is None:
        G = nx.Graph()  # 创建无向图
    for _, row in df.iterrows():
        tid = 't' + str(row['tweet_id'])
        G.add_node(tid) # 一次添加一个节点，字符串作为节点id
        G.nodes[tid]['tweet_id'] = True # right-hand side value is irrelevant for the lookup
        
        user_ids = row['user_mentions']
        user_ids.append(row['user_id'])
        user_ids = ['u_' + str(each) for each in user_ids]
        G.add_nodes_from(user_ids) # 添加多个节点
        for each in user_ids:
            G.nodes[each]['user_id'] = True 
        
        entities = row['entities']
#         words = ['e_' + each for each in entities]
        G.add_nodes_from(entities)
        for each in entities:
            G.nodes[each]['entities'] = True
        
        words = row['sampled_words']
        words = ['w_' + each for each in words]
        G.add_nodes_from(words)
        for each in words:
            G.nodes[each]['word'] = True
        
        edges =[]
        edges += [(tid, each) for each in user_ids]
        edges += [(tid, each) for each in entities]
        edges += [(tid, each) for each in words]
        G.add_edges_from(edges) # 同时添加多条边
    return G

In [19]:
# convert a heterogeneous social graph G to a homogeneous message graph following eq. 1 of the paper, 
# and store the sparse binary adjacency matrix of the homogeneous message graph.
def to_dgl_graph_v3(G, save_path=None):
    message = ''
    print('Start converting heterogeneous networks graph to homogeneous dgl graph.')
    message += 'Start converting heterogeneous networks graph to homogeneous dgl graph.\n'
    all_start = time()
    
    print('\tGetting a list of all nodes ...')
    message += '\tGetting a list of all nodes ...\n'
    start = time()
    all_nodes = list(G.nodes)
    mins = (time() -start) /60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\tGetting adjacency matrix ...')
    message += '\tGetting adjacency matrix ...\n'
    start = time()
    A = nx.to_numpy_matrix(G) # Returns the graph adjacency matrix as a Numpy matrix
    mins = (time() - start)/ 60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # compute commuting matrics
    print('\tGetting lists of nodes of various types ...')
    message += '\tGetting lists of nodes of various types ...\n'
    start = time()
    tid_nodes = list(nx.get_node_attributes(G, 'tweet_id').keys()) # get_node_attributes return node and its attributes
    userid_nodes = list(nx.get_node_attributes(G, 'user_id').keys())
    word_nodes = list(nx.get_node_attributes(G, 'word').keys())
    entity_nodes = list(nx.get_node_attributes(G, 'entity').keys())
    del G
    mins = (time() - start)/ 60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\tConverting node lists to index lists ...')
    message += '\tConverting node lists to index lists ...\n'
    start = time()
    # fine the index of target nodes in the list of all nodes
    indices_tid = [all_nodes.index(x) for x in tid_nodes]
    indices_userid = [all_nodes.index for x in userid_nodes]
    indices_word = [all_nodes.index(x) for x in word_nodes]
    indices_entity = [all_nodes.index(x) for x in entity_nodes]
    del tid_nodes
    del userid_nodes
    del word_nodes
    del entity_nodes
    mins = (time() -start)/60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # ----------------tweet-user-tweet------------------
    print('\tStart constructing tweet-user-tweet commuting matrix ...')
    print('\t\t\tStart constructing tweet-user matrix ...')
    message += '\tStart constructing tweet-user-tweet commuting matrix ...\n\t\t\tStart constructing tweet-user matrix ...\n'
    start = time()
    w_tid_userid = A[np.ix_(indices_tid, indices_userid)]  #生成一个open mash数组
    # return a N(indiced_tid)*N(indices_userid) matrix, representing the weight of edges between tid and userid
    mins = (time() - start)/60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # convert to scipy sparse matrix
    print('\t\t\tConverting to sparse matrix ...')
    message += '\t\t\tConverting to sparse matrix ...\n'
    start = time()
    s_w_tid_userid = sparse.csr_matrix(w_tid_userid) # matrix compression
    del w_tid_userid
    mins = (time() - start)/ 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tTransposing ...')
    message += '\t\t\tTransposing ...\n'
    start = time()
    s_w_userid_tid = s_w_tid_userid.transpose()
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tCalculating tweet-user * user-tweet ...')
    message += '\t\t\tCalculating tweet-user * user-tweet ...\n'
    start = time()
    s_m_tid_userid_tid = s_w_tid_userid * s_w_userid_tid # homogeneous message graph
    mins = (time() - start)/ 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tSaving ...')
    message += '\t\t\tSaving ...\n'
    start = time()
    if save_path is not None:
        sparse.save_npz(save_path + "s_m_tid_userid_tid.npz", s_m_tid_userid_tid)
        print('sparse binary userid commuting matrix saved.')
        del s_m_tid_userid_tid
    del s_w_tid_userid
    del s_w_userid_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # ----------tweet-ent-tweet-----------------
    print('\tStart constructing tweet-ent-tweet conmuting matrix ...')
    print('\t\t\tStart constructing tweet-ent matrix ...')
    message += '\tStart constructing tweet-ent-tweet commuting matrix ...\n\t\t\tStart constructing tweet-ent matrix ...\n'
    start = time()
    w_tid_entity = A[np.ix_(indices_tid, indices_entity)]
    mins = (time() - start) / 60
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # convert to scipy sparse matrix
    print('\t\t\tConverting to sparse matrix ...')
    message += '\t\t\tConverting to sparse matrix ...\n'
    start = time()
    s_w_tid_entity = sparse.csr_matrix(w_tid_entity)
    del w_tid_entity
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed : ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tTransposing ...')
    message += '\t\t\tTransposing ...\n'
    start = time()
    s_w_entity_tid = s_w_tid_entity.transpose()
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tCalculating tweet-ent * ent-tweet ...')
    message += '\t\t\tCalculating tweet-ent * ent-tweet ...\n'
    start = time()
    s_m_tid_entity_tid = s_w_tid_entity * s_w_entity_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tSaving ...')
    message += '\t\t\tSaving ...\n'
    start = time()
    if save_path is not None:
        sparse.save_npz(save_path + "s_m_tid_entity_tid.npz", s_m_tid_entity_tid)
        print('Sparse binary entity commuting matrix saved.')
        del s_m_tid_entity_tid
    del s_w_tid_entity
    del s_w_entity_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # --------tweet-word-tweet------------------
    print('\tStart constructing tweet-word-tweet commuting matrix ...')
    print('\t\t\tStart constructing tweet-word matrix ...')
    message +='\tStart constructing tweet-wrod-tweet commuting matrix ...\n\t\t\tStart constructing tweet-word matrix ...'
    start = time()
    w_tid_word = A[np.ix_(indices_tid, indices_word)]
    del A
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # convert to scipy sparse matrix
    print('\t\t\tConverting to Sparse matrix ...')
    message += '\t\t\tConverting to sparse matrix ...\n'
    start = time()
    s_w_tid_word = sparse.csr_matrix(w_tid_word)
    del w_tid_word
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tTransposing ...')
    message += '\t\t\tTransposing ...\n'
    start = time()
    s_w_word_tid = s_w_tid_word.transpose()
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tCalculating tweet-word * word-tweet ...')
    message += '\t\t\tCalculating tweet-word * word-tweet ...\n'
    start = time()
    s_m_tid_word_tid = s_w_tid_word * s_w_word_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    print('\t\t\tSaving ...')
    message += '\t\t\tSaving ...\n'
    start = time()
    if save_path is not None:
        sparse.save_npz(save_path + "s_m_tid_owrd_tid.npz", s_m_tid_word_tid)
        print("Sparse binary word commuting matrix saved.")
        del s_m_tid_word_tid
    del s_w_tid_word
    del s_w_word_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    
    # -----------compute tweet-tweet adjacency matrix --------
    print('\tComputing tweet-tweet adjacency matrix ...')
    message += '\tComputing tweet-tweet adjacency matrix ...\n'
    start = time()
    if save_path is not None:
        s_m_tid_userid_tid = sparse.load_npz(save_path + 's_m_tid_userid_tid.npz')
        print("Sparse binary userid commuting matrix loaded.")
        s_m_tid_entity_tid = sparse.load_npz(save_path + "s_m_tid_entity_tid.npz")
        print("Sparse binary entity commuting matrix loaded.")
        s_m_tid_word_tid = sparse.load_npz(save_path + "s_m_tid_word_tid.npz")
        print("Sparse binary word commuting matrix loaded.")
        
    s_A_tid_tid = s_m_tid_userid_tid + s_m_tid_entity_tid
    del s_m_tid_userid_tid
    del s_m_tid_entity_tid
    s_bool_A_tid_tid = (s_A_tid_tid + s_m_tid_word_tid).astype('bool')  # confirm the connect between tweets
    del s_m_tid_word_tid
    del s_A_tid_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    all_mins = (time() - all_start) / 60
    print('\tOver all time elapsed: ', all_mins, ' mins\n')
    message += '\tOver all time elapsed: '
    message += str(all_mins)
    message += ' mins\n'
    
    if save_path is not None:
        sparse.save_npz(save_path + "s_bool_A_tid_tid.npz", s_bool_A_tid_tid)
        print("Sparse binary adjacency matrix saved.")
        s_bool_A_tid_tid = sparse.load_npz(save_path + "s_bool_A_tid_tid.npz")
        print("Sparse binary adjacency matrix loaded.")
        
    # create correspoinding dgl graph
    G = dgl.DGLGraph(s_bool_A_tid_tid)
    print('We have %d nodes.' % G.number_of_nodes())
    print('We have %d edges' % G.number_of_edges())
    print()
    message += 'We have '
    message += str(G.number_of_nodes())
    message += ' nodes.'
    message += 'We have '
    message += str(G.number_of_edges())
    message += ' edges.\n'
    
    return all_mins, message

In [20]:
# Split the Twitter dataset by date into 21 message blocks, use the message blocks to construct heterogeneous social graphs,
# and maps them into homogeneous message graphs. 
# Note that:
# 1) We adopt the Latest Message Strategy (which is the most efficient and gives the strongest performance. See Section 4.4 of the paper for more details) here, 
# as a consequence, each message graph only contains the messages of the date and all previous messages are removed from the graph;
# To switch to the All Message Strategy or the Relevant Message Strategy, replace 'G = construct_graph_from_df(incr_df)' with 'G = construct_graph_from_df(incr_df, G)'.
# 2) For test purpose, set test=True, and the message blocks, as well as the resulted message graphs each will contain 100 messages.
# To use all the messages, set test=False, and the number of messages in the message blocks will follow Table. 4 of the paper.
def construct_incremental_dataset_0922(df, save_path, features, test=True):
    # If test equals true, construct the initial graph using test_ini_size_tweets
    # and increment the graph by test_incr_size tweets each day
    test_ini_size = 500
    test_incr_size = 100
    
    # save data splits for training/validate/test mask generation
    # data_split = []
    # save time spent for the heterogeneous -> homogeneous conversion of each graph
    all_graph_mins = []
    message = ''
    # extract distingct dates
    distinct_dates = df.date.unique() # 所有unique的date
    print('Number of distinct dates: ', len(distinct_dates))
    print()
    message += 'Number of distinct dates: '
    message += str(len(distinct_dates))
    message += '\n'
    
    # split data by dates and construct graphs
    # first week -> initial graph (20254 tweets)
    print('Start constructing initial graph ...')
    message += '\nStart constructing initial graph ...\n'
    ini_df = df
    G = construct_graph_from_df(ini_df)
    path = save_path + '0/'
    os.mkdir(path)  # 创建目录
    graph_mins, graph_message = to_dgl_graph_v3(G, save_path=path)  # convert a heterogeneous social graph to a homogeneous message graph
    message += graph_message
    print('Initial graph saved')
    message += 'Initial graph saved\n'
    # record the totoal number of tweets
    all_graph_mins.append(graph_mins)
    # extract and save the labels of corresponding tweets
    y = ini_df['event_id'].values
    y = [int(each) for each in y]
    np.save(path + 'labels.npy', np.asarray(y))  # ndarray对象，实际只创建一个指针
    print('Labels saved.')
    message += 'Labels saved.\n'
    # extract and sve the features of corresponding tweets
    indices = ini_df['index'].values.tolist()
    x = features[indices, :]
    np.save(path + 'features.npy', x)
    print('Features saved.')
    message += 'Features saved. \n\n'
    
    return message, all_graph_mins

In [21]:
save_path = './incremental_test/'
if not os.path.exists(save_path):
    os.mkdir(save_path)

# load data (68841 tweets, multiclasses filtered)
p_part1 = '../datasets/Twitter/68841_tweets_multiclasses_filtered_0722_part1.npy'
p_part2 = '../datasets/Twitter/68841_tweets_multiclasses_filtered_0722_part2.npy'
df_np_part1 = np.load(p_part1, allow_pickle=True)  # allow_pickle: 可选，布尔值，允许使用 Python pickles 保存对象数组，Python 中的 pickle 用于在保存到磁盘文件或从磁盘文件读取之前，对对象进行序列化和反序列化。
df_np_part2 = np.load(p_part2, allow_pickle=True)
df_np = np.concatenate((df_np_part1, df_np_part2), axis=0)
print('Data loaded.')
df = pd.DataFrame(data=df_np, columns=['event_id', 'tweet_id', 'text', 'user_id', 'created_at', 'user_loc',
                                      'place_type', 'place_full_name', 'place_country_code', 'hashtags',
                                      'user_mentions', 'image_urls', 'entities', 'words', 'filtered_words', 'sampled_words'])
print('Data converted to dataframe.')

# sort data by time
df = df.sort_values(by='created_at').reset_index()
# append data
df['data'] = [d.date() for d in df['created_at']]
# load features
# the dimension of feature is 300 in this dataset
f = np.load('../datasets/Twitter/features_69612_0709_spacy_lg_zero_multiclasses_filtered.npy')
# generate test graphs, features, and labels
message, all_graph_mins = construct_incremental_dataset_0922(df, save_path, f, True)
with open(save_path + 'node_edge_statistics.txt', 'w') as text_file:
    text_file.write(message)
np.save(save_path + 'all_graph_min.npy', np.asarray(all_graph_mins))
print('Time spent on heterogeneous -> homogeneous graph conversions: ', all_graph_mins)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/Twitter/68841_tweets_multiclasses_filtered_0722_part1.npy'

In [22]:
np.ix_([1,5,7,2],[0,3,1,2])

(array([[1],
        [5],
        [7],
        [2]]),
 array([[0, 3, 1, 2]]))

## utils

In [23]:
# utility，功能
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from torch.utils.data import Dataset

# 交集
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def run_kmeans(extract_features, extract_labels, indices, isoPath=None):
    # extract the features and labels of the test tweets
    if isoPath is not None:
        # Remove isolated points
        temp = torch.load(isoPath)
        temp = temp.cpu().detach().numpy()  # detach()阻断反向传播，返回值为tensor；numpy()将tensor转换为numpy
        non_isolated_index = list(np.where(temp != 1)[0]) # np.where返回符合条件元素的索引index
        indices = intersection(indices, non_isolated_index)
    # Extract labels
    extract_labels = extract_labels.cpu().numpy()
    labels_true = extract_labels[indices]
    
    # Extrac features
    X = extract_features.cpu().detach().numpy()
    assert labels_true.shape[0] == X.shape[0]  # assert断言，在判断式false时触发异常
    n_test_tweets = X.shape[0]  # 100
    
    # Get the total number of classes
    n_classes = len(ser(labels_true.tolist()))  # unique()和nunique()不香吗？
    
    # k-means clustering
    kmeans = KMeans(n_clusters=n_classes, random_state=0).fit(X)
    labels = kmeans.labels_
    
    nmi = metrics.normalized_mutual_info_score(labels_true, labels)  # 计算归一化互信息
    ami = metrics.adjusted_mutual_info_score(labels_true, labels)
    ari = metrics.adjusted_mutual_info_score(labels_true, labels)  # 计算兰德系数
    
    # Return number of test tweets, number of classes covered by the test tweets, and KMeans clustering NMI
    return n_test_tweets, n_classes, nmi, ami, ari

In [24]:
def evaluate(extract_features, extract_labels, indices, epoch, num_isolated_nodes, save_path,
             is_validation=True, cluster_type='kmeans'):
    message = ''
    message += '\nEpoch '
    message += str(epoch)
    message += '\n'
    
    # with isolated nodes
    if cluster_type == 'kmeans':
        n_tweets, n_classes, nmi, ami, ari = run_kmeans(extract_features, extract_labels, indices)
    elif cluster_type == 'dbscan':
        pass
    
    if is_validation:
        mode = 'validation'
    else:
        mode = 'test'
    message += '\tNumber of ' + mode + ' tweets: '
    message += str(n_tweets)
    message += '\n\tNumber of classes covered by ' + mode + ' tweets: '
    message += str(n_classes)
    message += '\n\t' + mode + ' NMI: '
    message += str(nmi)
    message += '\n\t' + mode + 'AMi: '
    message += str(ami)
    message += '\n\t' + mode + 'ARI'
    message += str(ari)
    if cluster_type == 'dbscan':
        message += '\n\t' + mode + ' best_eps: '
        message += '\n\t' + mode + ' best_min_Pts: '
    
    if num_isolated_nodes != 0:
        # without isolated nodes
        message += '\n\tWithout isolated nodes:'
        n_tweets, n_classes, nmi, ami, ari = run_kmeans(extract_features, extract_labels, indices, 
                                                       save_path + '/isolated_nodes.pt')
        message += '\tNumber of ' + mode + 'tweets: '
        message += str(n_tweets)
        message += '\n\tNumber of classes covered by ' + mode + ' tweets'
        message += str(n_classes)
        message += '\n\t' + mode + 'NMI: '
        message += str(nmi)
        message += '\n\t' + mode + 'AMI: '
        message += str(ami)
        message += '\n\t' + mode + 'ARI: '
        message += str(ari)
    message += '\n'
    
    with open(save_path + '/evaluate.txt', 'a') as f:
        f.write(message)
    print(message)
    
    np.save(save_path + '/%s_metric.npy' % mode, np.asarray([nmi, ami, ari]))
    return nmi

In [25]:
def generateMasks(length, data_split, train_i, i, validation_percent=0.2, save_path=None, remove_absolete=2):
    '''    
    Intro:
    This function generates train and validation indices for initial/maintenance epochs and test indices for inference(prediction) epochs
    If remove_obsolete mode 0 or 1:
    For initial/maintenance epochs:
    - The first (train_i + 1) blocks (blocks 0, ..., train_i) are used as training set (with explicit labels)
    - Randomly sample validation_percent of the training indices as validation indices
    For inference(prediction) epochs:
    - The (i + 1)th block (block i) is used as test set
    Note that other blocks (block train_i + 1, ..., i - 1) are also in the graph (without explicit labels, only their features and structural info are leveraged)
    If remove_obsolete mode 2:
    For initial/maintenance epochs:
    - The (i + 1) = (train_i + 1)th block (block train_i = i) is used as training set (with explicit labels)
    - Randomly sample validation_percent of the training indices as validation indices
    For inference(prediction) epochs:
    - The (i + 1)th block (block i) is used as test set
    :param length: the length of label list
    :param data_split: loaded splited data (generated in custom_message_graph.py)
    :param train_i, i: flag, indicating for initial/maintenance stage if train_i == i and inference stage for others
    :param validation_percent: the percent of validation data occupied in whole dataset
    :param save_path: path to save data
    :param num_indices_to_remove: number of indices ought to be removed
    :returns train indices, validation indices or test indices
    '''
    # step1: verify total number of nodes
    assert length == data_split[i] # 500
    
    # step2.0: if is in initial/maintenance epochs, generate train and validation indices
    if train_i == i:
        # step3: randomly shuffle the graph indices
        train_indices = torch.randperm(length)  # 返回一个随机打散的0-n-1数组
        # step4: get total number of validation indices
        n_validation_sample = int(length * validation_percent)
        # step5: sample n_validation_samples validation indices and use the rest as training indices
        validation__indices = train_indices[:n_validation_samples]
        train_indices = train_indices[n_validation_samples:]
        # step6: save indices
        if save_path is not None:
            torch.save(train_indices, save_path + '/train_indices.pt')
            torch.save(validation_indices, save_path + '/validation_indices.pt')
        return train_indices, validation__indices
    # step2.1: if is in inference(prediction) epochs, generate test indices
    else:
        test_indices = torch.arange(0, (data_split[i]), dtype=torch.long)
        if save_path is not None:
            torch.save(test_indices, save_path + '/test_indices.pt')
        return test_indices

In [26]:
def gen_offline_masks(length, validation_percent=0.2, test_percent=0.1):
    test_length = int(length * test_percent)
    valid_length = int(length * validation_percent)
    train_length = length - valid_length - test_length
    
    samples = torch.randperm(length)
    train_indices = samples[:train_length]
    valid_indices = samples[train_length: train_length + valid_length]
    test_indices = samples[train_length + valid_length:]
    
    return train_indices, valid_length, test_indices

In [27]:
def save_embeddings(extracted_features, save_path):
    torch.save(extracted_features, save_path + '/final_embeddings.pt')
    print('extracted features saved.')

In [28]:
a = np.array([2,43,2,42,12,454])[0]
np.where(a<5)

(array([0], dtype=int64),)

## gen_dataset

In [29]:
from scipy.sparse.coo import coo_matrix
from scipy.sparse.csr import csr_matrix
from scipy import sparse

from torch_geometric.data import Data
from torch_sparse.tensor import SparseTensor
# from .utils import generateMasks, gen_offline_masks，是指从utils.py文件中导入函数: generatemasks, gen_offline_masks.

In [30]:
def sparse_trans(datapath = 'incremental_0808/0/s_m_tid_userid_tid.npz'):
    relation = sparse.load_npz(datapath)
    all_edge_index = torch.tensor([], dtype=int)
    for node in range(relation.shape[0]):
        neighbor = torch.IntTensor(relation[node].toarray()).squeeze()  # IntTensor是torch定义的7中cpu tensor类型之一；
                                                                        # squeeze对数据维度进行压缩，删除所有为1的维度
        # del self_loop in advance
        neighbor[node] = 0
        neighbor_idx = neighbor.nonzero()  # 返回非零元素的索引
        neighbor_sum = neighbor_idx.size(0)  # 表示第0维度的数据量
        loop = torch.tensor(node).repeat(neighbor_sum, 1)  # repeat表示沿着指定的维度重复tensor的次数
        edge_index_i_j = torch.cat((loop, neighbor_idx), dim=1).t()  # cat表示拼接；t表示对二维矩阵进行转置
        self_loop = torch.tensor([[node], [node]])
        all_edge_index = torch.cat((all_edge_index, edge_index_i_j, self_loop), dim=1)
        del neighbor, neighbor_idx, loop, self_loop, edge_index_i_j
    return all_edge_index

In [31]:
def coo_trans(datapath = 'incremental_0808/0/s_m_tid_userid_tid.npz'):
    relation: csr_matrix = sparse.load_npz(datapath)
    relation: coo_matrix = relation.tocoo()
    sparse_edge_index = torch.LongTensor([relation.row, relation.col])  # sparse稀疏矩阵用三元组(row,col,data)来存储非零元素信息
    return sparse_edge_index

In [32]:
def create_dataset(loadpath, relation, mode):
    features = np.load(os.path.join(loadpath, str(mode[1]), 'features.npy'))
    features = torch.FloatTensor(features)
    print('features laoded')
    labels = np.load(os.path.join(loadpath, str(mode[1]), 'labels.npy'))
    print('labels loaded')
    labels = torch.LongTensor(labels)
    relation_edge_index = coo_trans(os.path.join(loadpath, str(mode[1]), 's_m_tid_%s_tid.npz' % relation))
    print('edge index laoded')
    data = Data(x=features, edge_index=relation_edge_index, y=labels)
    data_split = np.load(os.path.join(loadpath, 'data_split.npy'))
    train_i, i = mode[0], mode[1]
    if train_i == i:
        data.train_mask, data.val_mask = generateMasks(len(labels), data_split, train_i, i)
    else:
        data.test_mask = generateMasks(len(labels), data_split, train_i, i)
    return data

In [33]:
def create_homodataset(loadpath, mode, valid_percent=0.2):
    features = np.load(os.path.join(loadpath, str(mode[1]), 'features.npy'))
    features = torch.FloatTensor(features)
    print('features loaded')
    labels = np.load(os.path.join(loadpath, str(mode[1]), 'labels.npy'))
    print('labels loaded')
    labels = torch.LongTensor(labels)
    
    data = Data(x=features, edge_index=None, y=labels)  # torch_geometric提供的图数据类型Data，x表示tensor矩阵，
                                                        # 形状为[num_nodes, num_node_features]; edge_index表示coo格式的图的边关系，
                                                        # 形状为[2, num_edge]
    data_split = np.load(os.path.join(loadpath, 'data_split.npy'))
    train_i, i = mode[0], mode[1]
    if train_i == i:
        data.train_mask, data.val_mask = generateMasks(len(labels), data_split, train_i, i, valid_percent)
    else:
        data.test_mask = generateMasks(len(labels), data_split, train_i, i)
    return data

In [34]:
def create_offline_homodataset(loadpath, mode):
    features = np.load(os.path.join(loadpath, str(mode[1]), 'features.npy'))
    features = torch.FloatTensor(features)
    print('features loaded')
    labels = np.load(os.path.join(loadpath, str(mode[1]), 'labels.npy'))
    print('labels loaded')
    labels = torch.LongTensor(labels)
    data = Data(x=features, edge_index=None, y=labels)
    data.train_mask, data.val_mask, data.test_mask = generateMasks(len(labels))
    return data

In [35]:
def create_multi_relational_graph(loadpath, relations, mode):
    multi_relation_edge_index = [torch.load(loadpath + '/' + str(mode[1]), '/edge_index_%s.pt' % relatio) for relation in relations]
    print('sparse trans...')
    print('edge index loaded')
    

In [36]:
def save_multi_relational_graph(loadpath, relations, mode):
    for relation in relations:
        relation_edge_index = sparse_trans(os.path.join(loadpath, str(mode[1]), 's_m_tid_%s_tid.npz' % relation))
        torch.save(relation_edge_index, loadpath + '/' + str(mode[1]) + '/edge_index_%s.pt' % relation)

## step 3: save edge index

In [37]:
# acclerate the training process
import torch

data_path = 'incremental_cross_English_68841'
relation_ids = ['entity', 'userid', 'word']
for i in range(22):
    save_multi_relational_graph(data_path, relation_ids, [0,i])
    print('edge index saved')
print('all edge index saved')

FileNotFoundError: [Errno 2] No such file or directory: 'incremental_cross_English_68841\\0\\s_m_tid_entity_tid.npz'

## mysampler

In [42]:
import copy
from typing import List, Optional, Tuple, NamedTuple, Union, Callable
from scipy import sparse
import random

import torch
from torch import Tensor
from torch_sparse import SparseTensor
from torch_geometric.loader import NeighborSampler, RandomNodeSampler

In [44]:
class MySampler(object):
    def __init__(self, sampler) -> None:
        super().__init__()
        self.sampler = sampler
    
    def smaple(self, multi_relational_edge_index: List[Tensor], node_idx, sizes, batch_size):
        if self.sampler == 'RL_sampler':
            return self._RL_sample(multi_relational_edge_index, node_idx, sizes, batch_size)
        elif self.sampler == 'randdom_sampler':
            return self._random_sample(multi_relational_edge_index, node_idx, batch_size)
        elif self.sampler == 'const_sampler':
            return self._const_sample(multi_relational_edge_index, node_idx, batch_size)
    
    def _RL_sample(self, multi_relational_edge_index: List[Tensor], node_idx, sizes, batch_size):
        outs = []
        all_n_ids = []
        for id, edge_index in enumerate(multi_relational_edge_index):  # 返回数据和数据下标
            loader = NeighborSampler(edge_index=edge_index, sizes=sizes, node_idx=node_idx, return_e_id=False,
                                    batch_size=batch_size, num_workers=0)
            for id, (_, n_ids, adjs) in enumerate(loader):
                outs.append(adjs)
                all_n_ids.append(n_ids)
            
            assert id == 0
        return outs, all_n_ids
    
    def _random_sample(self, multi_relational_edge_index: List[Tensor], node_idx, batch_size):
        outs = []
        all_n_ids = []
        sizes = [random.randint(10,100), random.randint(10,50)]
        for edge_index in multi_relational_edge_index:
            loader = NeighborSampler(edge_index=edge_index, sizes=sizes, node_idx=node_idx, return_e_id=False,
                                    batch_size=batch_size, num_workers=0)
            for id, (_, n_ids, adjs) in enumerate(loader):
                outs.append(adjs)
                all_n_ids.append(n_ids)
            assert id == 0
        return outs, all_n_ids

    def _const_sample(self, multi_relational_edge_index: List[Tensor], node_idx, batch_size):
        outs = []
        all_n_ids = []
        sizes = [25, 15]
        for edge_index in multi_relational_edge_index:
            loader = NeighborSampler(edge_index=edge_index, sizes=sizes, node_idx=node_idx, return_e_id=False,
                                    batch_size=batch_size, num_workers=0)
            for id, (_, n_ids, adjs) in enumerate(loader):
                outs.append(adjs)
                all_n_ids.append(n_ids)
            assert id == 0
        return outs, all_n_ids

## metrics

In [128]:
import numpy as np

In [129]:
class Metric:
    def __init__(self):
        pass
    
    def __call__(self, outputs, target, loss):
        raise NotImplementedError
    
    def reset(self):
        raise NotImplementedError
    
    def value(self):
        raise NotImplementedError
    
    def name(self):
        raise NotImplementedError

In [103]:
class AccumulateAccuracy(Metric):
    '''
    works with classification model
    '''
    def __init__(self):
        self.correct = 0
        self.total = 0
    
    def __call__(self, outputs, target, loss):
        pred = outputs[0].data.max(1, keepdim=True)[1]
        self.correct += pred.eq(target[0].data.view_as(pred)).cpu().sum()
        self.total += target[0].size(0)
        return self.value()
    
    def reset(self):
        self.correct = 0
        self.total = 0
    
    def value(self):
        return 100 * float(self.correct) / self.total
    
    def name(self):
        return 'Accuracy'

In [104]:
class AverageNonzeroTripletsMetric(Metric):
    '''
    Counts average number of nonzero triplets found in minibatches
    '''
    def __init__(self):
        self.values = []
    
    def __call__(self, outputs, target, loss):
        self.values.append(loss[1])
        return self.value()
    
    def reset(self):
        self.values = []
    
    def value(self):
        return np.mean(self.values)
    
    def name(self):
        return 'Average nonzero triplets'

## stage

In [130]:
from typing import List, Any
import time
import torch.optim as optim
import gc  # garbage cleaning package

In [131]:
class FinEvent():
    def __init__(self, args) -> None:
        # register args
        self.args = args
    
    def inference(self,
                 train_i, i,
                 metrics,
                 embedding_save_path,
                 loss_fn,
                 model: MarGNN,
                 RL_thresholds=None,
                 loss_fn_dgi=None):
        # make dir for graph i
        # ./incremental_0808//embeddings_0403005348/block_xxx
        save_path_i = embedding_save_path + '/block_' + str(i)
        if not os.path.isdir(save_path_i):
            os.mkdir(save_path_i)
        
        # load data
        relation_ids: List[str] = ['entity', 'userid', 'word']
        homo_data = create_homodataset(self.args.data_path, [train_i, i], self.args.validation_percent)
        multi_r_data = create_multi_relational_graph(self.args.data_path, relation_ids, [train_i, i])
        num_relations = len(multi_r_data)
        
        device = torch.device('cuda' if torch.cuda.is_available() and self.args.use_cuda else 'cpu')
        
        # input ddimension (300 in our paper)
        features = homo_data.x
        feat_dim = features.size(1)
        
        # prepare graph configs for node filtering
        if self.args.is_initial:
            print('prepare node configures...')
            pre_node_dist(multi_r_data, homo_data.x, save_path_i)
            filter_path = save_path_i
        else:
            filter_path = self.args.data_path + str(i)
        
        if model is None:
            assert 'Cannot fine pre-trained model'
        
        # directly predict
        message = '\n-----------------Directly predict on block' + str(i) + '-----------------\n'
        print(message)
        print('RL Threshold using in this block:', RL_thresholds)
        
        model.eval()
        
        test_indices, labels = homo_data.test_mask, homo_data.y
        test_num_samples = test_indices.size(0)
        
        sampler = MySampler(self.args.sampler)
        
        # filter neighbor in advance to fit with neighbor sampling
        filtered_multi_r_data = RL_neighbor_filter(multi_r_data, RL_thresholds, filter_path) if RL_thresholds is not None and \
                                self.args.sampler == 'RL_sampler' else multi_r_data
        
        # batch testing
        extract_features = torch.FloatTensor([])
        num_batches = int(test_num_samples / self.args.batch_size) + 1
        with torch.no_grad():  # 在该模块下，所有计算得出的tensor的requires_grad都自动设置为False，不自动反向传播求导
            for batch in range(num_batches):
                start_batch = time.time()
                
                # split batch
                i_start = self.args.batch_size * batch
                i_end = min((batch + 1) * self.args.batch_size, test_num_samples)
                batch_nodes = test_indices[i_start:i_end]
                
                # sampling neighbors of batch nodes
                adjs, n_ids = sampler.sample(filtered_multi_r_data, node_idx= batch_nodes, sizes=[-1, -1], 
                                             batch_size= self.args.batch_size)
                pred = model(homo_data.x, adjs, n_ids, device, RL_thresholds)
                batch_seconds_spent = time.time() - start_batch
                
                # for we haven't shuffle the test indices(see utils.py)
                # the output embeddings can be simply stacked together
                extract_features = torch.cat((extract_features, pred.cpu().detach()), dim=0)
                
                del pred
                gc.collect()
        
        save_embeddings(extract_features, save_path_i)
        test_nmi = evaluate(extract_features,
                           labels,
                           indices=test_indices,
                           epoch=-1, # just for test
                           num_isolated_nodes=0,
                           save_path= save_path_i,
                           is_validation= False,
                           cluster_type= self.args.cluster_type)
        del homo_data, multi_r_data, features, filtered_multi_r_data
        torch.cuda.empty_cache()  # 释放显存
        
        return model
    
    # train on initial/maintenance graphs, t==0 or t % window_size == 0 in this paper
    def initial_maintain(self,
                        train_i, i,
                        metrics,
                        embedding_save_path,
                        loss_fn,
                        model=None,
                        loss_fn_dgi=None):
        '''
        :param i:
        :param data_split:
        :param metrics:
        :param embedding_save_path:
        :param loss_fn:
        :param model:
        :param loss_fn_dgi:
        :return:
        '''
        # make dir for graph i
        # ./incremental_0808//embeddings_0403005348/block_xxx
        save_path_i = embedding_save_path + '/block_' + str(i)
        if not os.path.isdir(save_path_i):
            os.mkdir(save_path_i)
        
        # load data
        relation_ids: List[str] = ['entity', 'userid', 'word']
        homo_data = create_homodataset(self.args.data_path, [train_i, i], self.args.validation_percent)
        multi_r_data = create_multi_relational_graph(self.args.data_path, relation_ids, [train_i, i])
        
        device = torch.device('cuda' if torch.cuda.is_available() and self.args.use_cuda else 'cpu')
        
        # input dimension (300 in our paper)
        num_dim = homo_data.x.size(0)
        feat_dim = homo_data.x.size(1)
        
        # prepare graph configs for node filtering
        if self.args.is_initial:
            print('prepare node %configures...')
            pre_node_dist(multi_r_data, homo_data.x, save_path_i)
            filter_path = save_path_i
        else:
            filter_path = self.args.data_path + str(i)
        
        if model is None: # pre-training stage in our paper
            # print('Pre-Train Stage')
            model = MarGNN((feat_dim, self.args.hidden_dim, self.args.out_dim, self.args.heads),
                          num_relations=num_relations, inter_opt=self.args.inter_opt, is_shared=self.args.is_shared)
        
        # define sampler
        sampler = MySampler(self.args.sampler)
        # load model to device
        model.to(device)
        
        # initialize RL thresholds
        # RL_threshold: [[.5], [.5], [.5]]
        RL_thresholds = torch.FloatTensor(self.args.threshold_start0)
        
        # define optimizer
        optimizer = optim.Adam(model.parameters(), lr=self.args.lr, weight_decay=1e-4)
        
        # record training log
        message = '\n------------- Start initial training / maintaining using block ' + str(i) + '----------\n'
        print(message)
        with open(save_path_i + '/log.txt', 'a') as f:
            f.write(message)
        
        # record the highest validation nmi ever got for early stopping
        best_vali_nmi = 1e-9
        best_epoch = 0
        wait = 0
        # record validation nmi of all epochs before early stop
        all_vali_nmi = []
        # recore the time spent in seconds on each batch of all training/maintaining epochs
        seconds_train_batches = []
        # record the time spent in mins on each epoch
        mins_train_epochs = []
        
        # step13: start training
        for epoch in range(self.args.n_epochs):
            start_epoch = time.time()
            losses = []
            totoal_loss = 0.0
            
            for metric in metrics:
                metic.reset()
                
            # Multi-Agent
            
            # filter neighbor in advance to fit with neighbor sampling
            filtered_multi_r_data = RL_neighbor_filter(multi_r_data, RL_thresholds, filter_path) if epoch >= self.args.RL_start0 and \
                                     self.args.sampler == 'RL_sampler' else multi_r_data
            model.train()
            train_num_samples, valid_num_samples = homo_data.train_mask.size(0), homo_data.vali_mask.size(0)
            all_num_samples = train_num_samples + valid_num_samples
            
            # batch training
            num_batches = int(train_num_samples / self.args.batch_size) + 1
            for batch in range(num_batches):
                start_batch = time.time()
                # split batch
                i_start = self.args.batch_size * batch
                i_end = min((batc + 1) * self.args.batch_size, train_num_samples)
                batch_nodes = homo_data.train_mask[i_start:i_end]
                batch_labels = homo_data.y[batch_nodes]
                
                # sampling neighobrs of batch nodes
                adjs, n_ids = sampler.sample(filtered_multi_r_data, node_ids=batch_nodes, sizes=[-1,-1], batch_size=self.args.batch_size)
                
                pred = model(homo_data.x, adjs, n_ids, device, RL_thresholds)
                
                extract_features = torch.cat((extract_features, pred.cpu().detach()), dim=0)
                
                del pred
                gc.collect()
            
            # save_embeddings(extract_reatures, save_path_i)
            # evaluate the model: conduct kMeans clustering on the validation and report NMI
            validation_nmi = evaluate(extract_features[homo_data.vali_mask],
                                     homo_data.y,
                                     epoch=epoch,
                                     num_isolated_nodes=0,
                                     save_path=save_path_i,
                                     is_validation=True,
                                     cluster_type=self.args.cluster_type)
            all_vali_nmi.append(validation_nmi)
            
            # step16: early stop
            if validation_nmi > best_vali_nmi:
                best_vali_nmi = validation_nmi
                best_epoch = epoch
                wait = 0
                # save model
                model_path = save_path_i + '/models'
                if (epoch == 0) and (not os.path.isdir(model_path)):
                    os.mkdir(model_path)
                p = model_path + '/best.pt'
                torch.save(model.state_dict(), p)  # 保存模型，OrderDict存储网络结构的名字和对应的参数
                print('Best model saved after epoch ', str(epoch))
            else:
                wait += 1
            if wait >= self.args.patience:
                print('Saved all_mins_spent')
                print('Early stopping at epoch ', str(epoch))
                print('Best model was at epoch ', str(best_epoch))
                break
            # end one epoch
        
        # save all validation mi
        np.save(save_path_i + '/all_vali_nmi.npy', np.asarray(all_vali_nmi))
        # save time spent on epochs
        np.save(save_path_i + '/mins_train_epochs.npy', np.asarray(mins_train_epochs))
        print('Saved mins_train_epochs')
        # save time spent on batches
        np.save(save_path_i + '/seconds_train_batches.npy', np.asarray(seconds_train_batches))
        print('Best model loaded.')
        
        del homo_data, multi_r_data
        torch.cuda.empty_cache()  # 释放显存
        
        return model, RL_thresholds

# FinEvent Incremental

In [ ]:
'''
step 1. run utils/generate_initial_features.py to generate the initial features for the messages

step 2. run utils/custom_message_graph.py to construct incremental message graphs. To construct small message graphs for test purpose, set test=True when calling construct_incremental_dataset_0922(). To use all the messages (see Appendix of the paper for a statistic of the number of messages in the graphs), set test=False.

step 3. run utils/save_edge_index.py in advance to acclerate the training process.

step 4. run main.py
'''

In [105]:
import numpy as np
import json  # 轻量级的数据交换格式，易于阅读和编写
import argparse  # 用于更方便地进行超参数的保存和修改
import torch

from time import localtime, strftime

In [106]:
def args_register():
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_epochs', default=50, type=int, help='Number of initial-training/maintenance-training epochs.')
    parser.add_argument('--window_size', default=3, type=int, help='Maintain the model after predicting window_size blocks.')
    parser.add_argument('--patience', default=5, type=int, help='Early stop if performance did not improve in the last patience epochs.')
    
    parser.add_argument('--margin', default=3, type=float, help='Margin for computing triplet losses')
    parser.add_argument('--lr', default=1e-3, type=float, help='Learning rate')
    parser.add_argument('--batch_size', default=100, type=int, help='Batch size (number of nodes sampled to compute triplet loss in each batch)')
    
    parser.add_argument('--hidden_dim', default=128, type=int, help='Hidden dimension')
    parser.add_argument('--out_dim', default=64, type=int, help='Output dimension of tweet representations')
    parser.add_argument('heads', default=4, type=int, help='Number of heads used in GAT')
    parser.add_argument('--validation_percent', default=0.2, type=float, help='Percentage of validation nodes(tweets)')
    parser.add_argument('--use_hardest_neg', dest='user_hardes_neg', default=False, action='store_true', 
                       help='If true, use hardest negative messages to form triplets. Otherwise use random ones')
    parser.add_argument('--is_shared', default=False)
    parser.add_argument('--inter_opt', default='cat_w_avg')
    parser.add_argument('--is_initial', default=False)
    parser.add_argument('--sampler', default='RL_sampler')
    parser.add_argument('--cluster_type', default='kmeans', help='Types of clustering algorithms') # DBSCAN
    
    # RL-0
    parser.add_argument('--threshod_start0', default=[[0.2],[0.2],[0.2]], type=float, 
                        help='The initial value of filter threshold for state1 or state3')
    parser.add_argument('--RL_step0', default=0.02, type=float, help='The step size of RL for state1 or state3')
    parser.add_argument('--RL_start0', default=0, type=int, help='The starting epoch of RL for state1 or state3')
    
    # RL-1
    parser.add_argument('--eps_start', default=0.001, type=float, help='The initial value of the eps for state2')
    parser.add_argument('--eps_step', default=0.02, type=float, help='The step size of eps for state2')
    parser.add_argument('--min_Pts_start', default=2, type=int, help='The initial value of the min_Pts for state2')
    parser.add_argument('--min_Pts_step', default=1, type=int, help='The step size of min_Pts for state2')
    
    # other arguments
    parser.add_argument('--user_cuda', dest='use_cuda', default=True, action='store_true', help='Use cuda')
    parser.add_argument('--data_path', default='./incremental_0502/', type=str, help='Path of features, labels and edges')
    # format: './incremental_0808/incremental_graphs_0808/embeddings_XXXX'
    parser.add_argument('--mask_path', default=None, type=str, help='File path that contains the training, validation and test masks')
    # format: './incremental_0808/incremental_graphs_0808/embeddings_XXXX'
    parser.add_argument('--log_interval', default=10, type=int, help='Log interval')
    args = parser.parse_args()  # 解析参数
    
    return args

In [107]:
if __name__ == '__main__':
    # define args
    args = args_register()
    
    # check CUDA
    print('Using CUDA:', torch.cuda.is_available())
    
    # create working path
    embedding_save_path = args.data_path + '/embeddings_' + strftime("%m%d%H%M%S", localtime())
    os.mkdir(embedding_save_path)
    print('embedding save path: ', embedding_save_path)
    
    # record hyper-parameters
    with open(embedding_save_path + '/args.txt', 'w') as f:
        json.dump(args.__dict__, f, indent=2)
    
    print('Batch Size:', args.batch_size)
    print('Intra Agg Mode:', args.is_shared)
    print('Inter Agg Mode:', args.inter_opt)
    print('Reserve node config?', args.is_initial)
    
    # load number of messages in each blocks
    # e.g. data_split = [  500  ,   100, ...,  100]
    #                    block_0  block_1    block_n
    data_split = np.load(args.data_path + '/data_split.npy')
    
    # define loss function
    # contrastive loss in our paper
    if args.use_hardest_neg:
        loss_fn = OnlineTripletLoss(args.margin, HardestNegativeTripletSelector(args.margin))
    else:
        loss_fn = OnlineTripletLoss(args.margin, RandomNegativeTripletSelector(args.margin))
        
    # define metrics
    BCL_metrics = [AverageNonzeroTriplesMetric()]
    
    # define detection stage
    Streaming = FinEvent(args)
    
    # pre-train stage: train on initial graph
    train_i = 0
    model, RL_thresholds = Streaming.initial_maintain(train_i = train_i,
                                                     i = 0,
                                                     metrics = BCL_metrics, 
                                                     embedding_save_path = embedding_save_path,
                                                     loss_fn = loss_fn,
                                                     model = None)
    
    # detection-maintenance stage: incremental training and detection
    for i in range(1, data_split.shape[0]):
        # infer every block
        model = Streaming.inference(train_i=train_i,
                                   i=i,
                                   metrics=BCL_metrics,
                                   embedding_save_path=embedding_save_path,
                                   loss_fn=loss_fn,
                                   model=model,
                                   RL_thresholds=RL_thresholds)
        
        # maintenance in window size and desert the last block
        if i % args.window_size == 0 and i != data_split.shape[0] -1:
            model, RL_thresholds = Streaming.initial_maintain(train_i=train_i,
                                                             i=i,
                                                             metrics=BCL_metrics,
                                                             embedding_save_path=embedding_save_path,
                                                             loss_fn=loss_fn,
                                                             model=None)

usage: ipykernel_launcher.py [-h] [--n_epochs N_EPOCHS] [--window_size WINDOW_SIZE] [--patience PATIENCE]
                             [--margin MARGIN] [--lr LR] [--batch_size BATCH_SIZE] [--hidden_dim HIDDEN_DIM]
                             [--out_dim OUT_DIM] [--validation_percent VALIDATION_PERCENT] [--use_hardest_neg]
                             [--is_shared IS_SHARED] [--inter_opt INTER_OPT] [--is_initial IS_INITIAL]
                             [--sampler SAMPLER] [--cluster_type CLUSTER_TYPE] [--threshod_start0 THRESHOD_START0]
                             [--RL_step0 RL_STEP0] [--RL_start0 RL_START0] [--eps_start EPS_START]
                             [--eps_step EPS_STEP] [--min_Pts_start MIN_PTS_START] [--min_Pts_step MIN_PTS_STEP]
                             [--user_cuda] [--data_path DATA_PATH] [--mask_path MASK_PATH]
                             [--log_interval LOG_INTERVAL]
                             heads
ipykernel_launcher.py: error: argument heads: invalid int v

AssertionError: 

# FinEvent Offline

In [ ]:
'''
step 1. run utils/generate_initial_features.py to generate the initial features for the messages

step 2. run utils/custom_message_graph.py to construct incremental message graphs. To construct small message graphs for test purpose, set test=True when calling construct_incremental_dataset_0922(). To use all the messages (see Appendix of the paper for a statistic of the number of messages in the graphs), set test=False.

step 3. run utils/save_edge_index.py in advance to acclerate the training process.

step 4. run offline.py
'''

In [108]:
import numpy as np
import json
import argparse
import torch
from time import localtime, strftime  # strftime() 函数用于格式化时间，返回以可读字符串表示的当地时间

import os
import torch.optim as optim
import gc
import time 
from typing import List, Any

In [120]:
def args_register():
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_epochs', default=50, type=int, help='Number of initial-training/maintenance-training epochs.')
    parser.add_argument('--window_size', default=3, type=int, help='Maintain the model after predicting window_size blocks.')
    parser.add_argument('--patience', default=5, type=int, help='Early stop if perfermance did not improve in the last patience epochs.')
    parser.add_argument('--margin', default=3, type=float, help='Margin for computing triplet losses')
    parser.add_argument('--lr', default=1e-3, type=float, help='Learning rate')
    
    parser.add_argument('--batch_size', default=100, type=int, help='Batch size (number of nodes sampled to compute triplet loss in each batch)')
    parser.add_argument('--hidden_dim', default=128, type=int, help='Hidden dimension')
    parser.add_argument('--out_dim', default=64, type=int, help='Output dimension of tweet representation')
    parser.add_argument('--heads', default=4, type=int, help='Number of heads used in GAT')
    parser.add_argument('--validation_percent', default=0.2, type=float, help='Percentage of validation nodes(tweets)')
    parser.add_argument('--use_hardest_neg', dest='use_hardest_neg', default=False, action='store_true', 
                        help='If true, use hardest negative messages to form triplets. Otherwise use random ones')
    parser.add_argument('--is_shared', default=False)
    parser.add_argument('--inter_opt', default='cat_w_avg')
    parser.add_argument('--is_initial', default=False)
    parser.add_argument('--sampler', default='RL_sampler')
    parser.add_argument('--cluster_type', default='kmeans', help='Types of clustering algorithms') # DBSCAN
    
    # RL-0
    parser.add_argument('--threshold_start0', default=[[0.2],[0.2],[0.2]], type=float, 
                        help='The initial value of the filter threshold for state1 or state3')
    parser.add_argument('--RL_step0', default=0.02, type=float, help='The starting epoch of RL for state1 or state3')
    parser.add_argument('--RL-start0', default=0, type=int, help='The starting epoch of RL for state1 or state3')
    
    # RL-1
    parser.add_argument('--eps_start', default=0.001, type=float, help='The initial value of the eps for state2')
    parser.add_argument('--eps_step', default=0.02, type=float, help='The step size of eps for state2')
    parser.add_argument('--min_Pts_start', default=2, type=int, help='The initial value of the min_Pts for state2')
    parser.add_argument('--min_Pts_step', default=1, type=int, help='The step size of min_Pts for state2')
    
    # other arguments
    parser.add_argument('--use_cuda', dest='use_cuda', default=True, action='store_true', help='Use cuda')
    parser.add_argument('--data_path', default='./incremental_0502/', type=str, help='Path of features, labels and edges')
    # format: './incremental_0808/incremental_graphs_0808/embeddings_XXXX'
    parser.add_argument('--mask_path', default=None, type=str, help='File path that contains the training, validation and test masks')
    # format: './incremental_0808/incremental_graphs_0808/embeddings_XXXX'
    parser.add_argument('--log_interval', default=10, type=int, help='Log interval')
    
    args = parser.parse_args()  # 解析参数
    
    return args


In [118]:
def offline_stage(train_i, i,
                 args,
                 metrics,
                 embedding_save_path,
                 loss_fn,
                 model=None,
                 loss_fn_dgi=None):
    # step1: make dir for graph i
    # ./incremental_0808//embeddings_0403005348/block_xxx
    save_path_i = embedding_save_path + '/block_' + str(i)
    if not os.path.isdir(save_path_i):
        os.mkdir(save_path_i)
    
    # step2: load data
    realtion_ids: List[str] = ['entity', 'userid', 'word']
    homo_data = create_offline_homodataset(args.data_path, [train_i, i])
    multi_r_data = create_multi_relational_graph(args.data_path, relation_ids, [train_i, i])
    num_relations = len(multi_r_data)
    
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_cuda else 'cpu')
    
    # input dimension (300 in our paper)
    num_dim = homo_data.x.size(0)
    feat_dim = homo_data.x.size(1)
    
    # prepare graph configs for node filtering
    if args.is_initial:
        print('prepare node configures...')
        pre_node_dist(multi_r_data, homo_data.x, save_path_i)
        filter_path = save_path_i
    else:
        filter_path = args.data_path + str(i)
    
    if model is None:  # pre-training stage in our paper
        # print('Pre-Train Stage...')
        model = MarGNN((feat_dim, args.hidden_dim, args.out_dim, args.heads), 
                      num_relations=num_relations, inter_opt=args.inter_opt,is_shared=args.is_shared)
    
    # define sampler
    sampler = MySampler(args.sampler)
    # load model to device
    model.to(device)
    
    # initialize RL thresholds
    # RL_threshold: [[.5],[.5],[.5]]
    RL_thresholds = torch.FloatTensor(args.threhold_start0)
    
    # define optimizer
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)
    
    # record training log
    message = '\n------Start initial training /maintaining using block' + str(i) + '------\n'
    print(message)
    with open(save_path_i + '/log.txt', 'a') as f:
        f.write(message)
    
    # step12.0: record the highest validation nmi ever got for early stopping
    best_vali_nmi = 1e-9
    best_epoch = 0
    wait = 0
    # step12.1: record validation nmi of all epochs before early stop
    all_vali_nmi = []
    # step12.2: record the time spent in seconds on each batch of all training/maintaining epochs
    seconds_train_batches = []
    # step12.3: record the time spent in mins on each epoch
    mins_train_epochs = []
    
    # step13: start training 
    for epoch in range(args.n_epochs):
        start_epoch = time.time()
        losses = []
        total_loss = 0.0
        
        for metric in metrics:
            metric.reset()
        
        # Multi-Agent
        
        # filter neighbor in adbvance to fit with neighbor sampling
        filtered_multi_r_data = RL_neighbor_filter(multi_r_data, RL_thresholds, filter_path) if epoch >= args.RL_start0 and \
                                args.sampler == 'RL_sampler' else multi_r_data
        # step13.0: forward
        model.train()
        
        train_num_samples, valid_num_samples, test_num_samples = homo_data.train_mask.size(0), homo.val_mask.size(0), homo_data.test_mask.size(0)
        all_num_samples = train_num_samples + valid_num_samples + test_num_samples
        
        torch.save(homo_data.train_mask, save_path_i + '/train_mask.pt')
        torch.save(homo_data.val_mask, save_path_i + '/valid_mask.pt')
        torch.save(homo_data.test_mask, save_path_i + '/test_mask.pt')
        
        # batch training
        num_batches = int(train_num_samples / args.batch_size) + 1
        for batch in range(num_batches):
            start_batch = time.time()
            # split batch
            i_start = args.batch_size * batch
            i_end = min((batch + 1) * args.batch_size, train_num_samples)
            batch_nodes = homo_data.train_mask[i_start:i_end]
            batch_labels = homo_data.y[batch_nodes]
            
            # sampling neighbors of batch nodes
            adjs, n_ids = sampler.sample(filtered_multi_r_data, node_idx=batch_nodes, size=[-1,-1],batch_size=args.batch_size)
            
            optimizer.zero_grad()  # 将参数置0
            
            pred = model(homo_data.x, adjs, n_ids, device, RL_thresholds)
            
            loss_outputs = loss_fn(pred, batch_labels)
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            losses.append(loss.item())
            total_loss += loss.item()
            
            # step13.1: metrics
            for metric in metrics:
                metric(pred, batch_labels, loss_outputs)
            if batch % args.log_interval == 0:
                message = 'Train: [{}/{} ({:.0f}%)] \tloss: {:.6f}'.format(batch * args.batch_size, train_num_samples,
                          100. * batch / ((train_num_samples // args.batch_size) + 1), np.mean(losses))
                for metric in metrics:
                    message += '\t{}: {:.4f}'.format(metric.name(), metric.value())
                # print(message)
                with open(save_path_i + '.log.txt', 'a') as f:
                    f.write(message)
                losses = []
            
            # print(torch.cuda.memory_summary(device=None, abbreviated=False))
            del pred, loss_outputs
            gc.collect()
            
            # step13.2: backward
            loss.backward()
            optimizer.step()
            
            batch_seconds_spent = time.time() - start_batch
            seconds_train_batches.append(batch_seconds_spent)
            
            del loss
            gc.collect()
        
        # step14: print loss
        total_loss /= (batch + 1)
        message = 'Epoch: {}/{}. Average loss: {:.4f}'.format(epoch, args.n_epochs, total_loss)
        
        for metric in metrics:
            message += '\t{}: {:.4f}'.format(metric.name(), metric.value())
        mins_spent = (time.time() - start_epoch) / 60
        message += '\nThis epoch took {:.2f} mins'.format(mins_spent)
        message += '\n'
        print(message)
        with open(save_path_i + '/log.txt', 'a') as f:
            f.write(message)
        mins_train_epochs.append(mins_spent)
        
        # step15: validation
        # inder the representations of all tweets
        model.eval()
        
        # we recommend to forward all nodes and select the validation indices instead
        extract_features = torch.FloatTensor([])
        num_batches = int(all_num_samples / args.batch_size) + 1
        
        # all mask are then splited into mini-batch in order
        all_mask = torch.arange(0, num_dim, dtype=torch.long)
        
        for batch in range(num_batches):
            start_batch = time.time()
            
            # split batch
            i_start = args.batch_size * batch
            i_end = min((batch + 1) * args.batch_size, all_num_samples)
            batch_nodes = all_mask[i_start:i_end]
            
            # sampling neighbors of batch nodes
            adjs, n_ids = sampler.sample(filtered_multi_r_data, node_idx=batch_nodes, sizes=[-1,-1], batch_size=args.batch_size)
            
            pred = model(homo_data.x, adjs, n_ids, device, RL_thresholds)
            
            extract_features = torch.cat((extract_features, pred.cpu().detach()), dim=0)
            
            del pred
            gc.collect()
        
        # evaluate the model: conduct kMeans clustering on the validation and report NMI
        validation_nmi = evaluate(extract_features[homo_data.val_mask],
                                 homo_data.y,
                                 indices=homo_data.val_mask,
                                 epoch=epoch,
                                 num_isolated_nodes=0,
                                 save_path=save_path_i,
                                 is_validation=True,
                                 cluster_type=args.cluster_type)
        all_vali_nmi.append(validation_nmi)
        
        # step16: early stop
        if validation_nmi > best_vali_nmi:
            best_vali_nmi = validation_nmi
            best_epoch = epoch
            wait = 0
            # save model
            model_path = save_path_i + '/models'
            if (epoch == 0) and (not os.path.isdir(model_path)):
                os.mkdir(model_path)
            p = model_path + '/best.pt'
            torch.save(model.stage_dict(), p)
            print('Best model was at epoch ', str(best_epoch))
        else:
            wait += 1
        if wait >= args.patience:
            print('Saved all_mins_spent')
            print('Early stopping at epoch ', str(epoch))
            print('Best model was at epoch ', str(best_epoch))
            break
        # end one epoch
    
    # step17: save all validation nmi
    np.save(save_path_i + '/all_vali_nmi.npy', np.asarray(all_vali_nmi))
    # save time spent on epochs
    np.save(save_path_i + '/mins_train_epochs.npy', np.asarray(mins_train_epochs))
    print('Saved mins_train_epochs.')
    # save time spent on batches
    np.save(save_path_i + '/seconds_train_batches.npy', np.asarray(seconds_train_batches))
    print('Saved seconds_train_batches.')
    
    # step18: load the best model of the current block
    best_model_path = save_path_i + '/models/best.pt'
    model.load_state_dict(torch.load(best_model_path))
    print('Best model loaded.')
    
    # del homo_data, multi_r_data
    torch.cuda.empty_cache()
    
    # test
    model.eval()
    
    # we recommend to forward all nodes and select the validation indices instead
    extract_features = torch.FloatTensor([])
    num_batches = int(all_num_samples /args.batch_size) + 1
    
    # all mask are then splited into mini-batch in order
    all_mask = torch.arange(0, num_dim, dtype= torch.long)
    
    for batch in range(num_batches):
        start_batch = time.time()
        
        # split batch
        i_start = args.batch_size * batch
        i_end = min((batch +1) * args.batch_size, all_num_samples)
        batch_nodes = all_mask[i_start:i_end]
        batch_labels = homo_data.y[batch_nodes]
        
        # sampling neighbors of batch nodes
        adjs, n_ids = sampler.sample(filtered_multi_r_data, node_idx=batch_nodes, sizes=[-1,-1], batch_size=args.batch_size)
        
        pred = model(homo_data.x, adjs, n_ids, device, RL_thresholds)
        
        extract_features = torch.cat((extract_features, pred.cpu().detach()), dim=0)
        del pred
        gc.collect()
        
    save_embeddings(extract_features, save_path_i)
    
    test_nmi = evaluate(extract_features[homo_data.test_mask],
                       homo_data.y,
                       indices=homo_data.test_mask,
                       epoch=-1,
                       num_isolated_nodes=0,
                       save_path=save_path_i,
                       is_validation=True,
                       cluster_type=args.cluster_type)
                                     

In [121]:
if __name__ == '__main__':
    # define args
    args = args_register()
    
    # check CUDA
    print('Using CUDA:', torch.cuda.is_available())
    
    # create working path
    embedding_save_path = args.data_path + '/embeddings_' + strftime('%m%d%H%M%S', localtime())
    os.mkdir(embedding_save_path)
    print('embedding save path: ', embedding_save_path)
    
    # record hyper-parameters
    with open(embedding_save_path + '/args.txt', 'w') as f:
        json.dump(args.__dict__, f, indent=2)
    
    print('Batch Size:', args.batch_size)
    print('Intra Agg Mode:', args.is_shared)
    print('Inter Agg Mode:', args.inter_opt)
    print('Reserve node config?', args.is_initial)
    # load number of message in each blocks
    # e.g. data_split = [  500  ,   100, ...,  100]
    #                    block_0  block_1    block_n
    
    # define loss function
    # contrastive loss in our paper 
    if args.use_hardest_neg:
        loss_fn = OnlineTripletLoss(args.margin, HardestNegativeTripletSelector(args.margin))
    else:
        loss_fn = OnlineTripletLoss(args.margin, RandomNegativeTripletSelector(args.margin))
    
    # define metrics
    BCL_metrics = [AverageNonzeroTripletsMetric()]
    
    # define detection stage
    Streaming = FinEvent(args)
    
    # pre-train stage: train on initial graph
    train_i = 0
    model, RL_thresholds = offline_stage(train_i=train_i,
                                        args=args,
                                        i=0,
                                        metrics=BCL_metrics,
                                        embedding_save_path=embedding_save_path,
                                        loss_fn=loss_fn,
                                        model=None)

usage: ipykernel_launcher.py [-h] [--n_epochs N_EPOCHS] [--window_size WINDOW_SIZE] [--patience PATIENCE]
                             [--margin MARGIN] [--lr LR] [--batch_size BATCH_SIZE] [--hidden_dim HIDDEN_DIM]
                             [--out_dim OUT_DIM] [--heads HEADS] [--validation_percent VALIDATION_PERCENT]
                             [--use_hardest_neg] [--is_shared IS_SHARED] [--inter_opt INTER_OPT]
                             [--is_initial IS_INITIAL] [--sampler SAMPLER] [--cluster_type CLUSTER_TYPE]
                             [--threshold_start0 THRESHOLD_START0] [--RL_step0 RL_STEP0] [--RL-start0 RL_START0]
                             [--eps_start EPS_START] [--eps_step EPS_STEP] [--min_Pts_start MIN_PTS_START]
                             [--min_Pts_step MIN_PTS_STEP] [--use_cuda] [--data_path DATA_PATH]
                             [--mask_path MASK_PATH] [--log_interval LOG_INTERVAL]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\yysgz\AppDa

SystemExit: 2

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# FinEvent Cross-lingual

In [122]:
import numpy as np
import json
import argparse
import torch
from time import localtime, strftime

In [124]:
def args_register():
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_epochs', default=50, type=int, help='Number of initial-training/maintenance-training epochs.')
    parser.add_argument('--window_size', default=3, type=int, help='Maintain the model after predicting window_size blocks.')
    parser.add_argument('--patience', default=5, type=int, help='Early stop if performance did not improve in the last patience epochs.')
    parser.add_argument('--margin', default=3., type=float, help='Margin for computing triplet losses')
    parser.add_argument('--lr', default=1e-3, type=float, help='Learning rate')
    
    parser.add_argument('--batch_size', default=100, type=int, help='Batch size (number of nodes sampled to compute triplet loss in each batch)')
    parser.add_argument('--hidden_dim', default=128, type=int, help='Hidden dimension')
    parser.add_argument('--out_dim', default=64, type=int, help='Output dimension of tweet representations')
    parser.add_argument('--heads', default=4, type=int, help='Number of heads used in GAT')
    parser.add_argument('--validation_percent', default=0.2, type=float, help='Percentage of validation nodes(tweets)')
    
    parser.add_argument('--use_hardest_neg', dest='use_hardest_neg', default=False, action='store_true', 
                       help='If true, use hardest negative messages to form triplets. Otherwise use random ones')
    parser.add_argument('--is_shared', default=False)
    parser.add_argument('--inter_opt', default='cat_w_avg')
    parser.add_argument('--is_initial', default=False)
    parser.add_argument('--sampler', default='RL_sampler')
    parser.add_argument('--cluster_type', default='kmeans', help='Types of clustering algorithms')  # DBSCAN
    
    # RL-0
    parser.add_argument('--threshold_start0', default=[[0.5],[0.5],[0.5]], type=float, 
                        help='The initial value of the filter threshold for state1 or state3')
    parser.add_argument('--RL_step0', default=0.02, type=float, help='The step size of RL for state1 or state3')
    parser.add_argument('--RL_start0', default=0, type=int, help='The starting epoch of RL for state1 or state3')
    
    # RL-1
    parser.add_argument('--eps_start', default=0.001, type=float, help='The initial value of the eps for state2')
    parser.add_argument('--eps_step', default=0.02, type=float, help='The step size of eps for state2')
    parser.add_argument('--min_Pts_start', default=2, type=int, help='The initial value of the min_Pts for state2')
    parser.add_argument('--min_Pts_step', default=1, type=int, help='The step size of min_Pts for state2')
    
    # other arguments
    parser.add_argument('--use_cuda', dest='use_cuda', default=True, action='store_true', help='Use cuda')
    parser.add_argument('--data_path', default='./incremental_0502/', type=str, help='Path of features, labels and edges')
    # format: './incremental_0808/incremental_graphs_0808/embeddings_XXXX'
    parser.add_argument('--mask_path', default=None, type=str, help='File path that contains the training, validation and test masks')
    # format: './incremental_0808/incremental_graphs_0808/embeddings_XXXX'
    parser.add_argument('--resume_path', default='incremental_cross_English_68841/', type=str, 
                       help='Resume trained model and directly used to inference')
    parser.add_argument('--log_interval', default=10, type=int, help='Log interval')
    args = parser.parse_args()  # 解析参数
    
    return args

In [126]:
if __name__ == '__main__':
    # define args
    args = args_register()
    
    # check CUDA
    print('Using CUDA:', torch.cuda.is_available())
    
    # create working path
    embedding_save_path = args.data_path + '/embeddings_' + strftime('%m%d%H%M%S', localtime())
    os.mkdir(embedding_save_path)
    print('embedding save path:', embedding_save_path)
    
    print('Batch size: ', args.batch_size)
    print('Intra Agg Mode: ', args.is_shared)
    print('Inter Agg Mode: ', args.inter_opt)
    print('Reserve node config?', args.is_initial)
    
    print('Trained model from %s dataset' % args.resume_path)
    print('Inference dataset: ', args.data_path)
    
    # record hyper=parameters
    with open(embedding_save_path + '/args.txt', 'w') as f:
        json.dump(args.__dict__, f, indent=2)
        
    # load number of messages in each blocks
    # e.g. data_split = [  500  ,   100, ...,  100]
    #                    block_0  block_1    block_n
    data_split = np.load(args.data_path + '/data_split.npy')
    # define loss function
    # contrastive loss in our paper
    if args.use_hardest_neg:
        loss_fn = OnlineTripletLoss(args.margin, HardestNegativeTripletSelector(args.margin))
    else:
        loss_fn = OnlineTripletLoss(args.margin, RandomNegativeTripletSelector(args.margin))
    
    # define metrics
    BCL_metrics = [AverageNonzeroTripletsMetric()]
    
    # define detection stage
    Streaming = FinEvent(args)
    
    # pre-train stage: train on initial graph
    train_i = 0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MarGNN((302, args.hidden_dim, args.out_dim, args.heads), 
                   num_relations=3, inter_opt=args.inter_opt, is_shared=args.is_shared)
    model.load_state_dict(torch.laod(args.resume_path + '/block_18/models/best.pt'))
    model, RL_thresholds = Streaming.initial_maintain(train_i=train_i,
                                                     i=0,
                                                     metrics=BCL_metrics,
                                                     embedding_save_path=embedding_save_path,
                                                     loss_fn=loss_fn,
                                                     model=None)
    
    # detection-maintenance stage: incremental training and detection
    for i in range(1, data_split.shape[0]):
        # infer every block
        model = Streaming.inference(train_i=train_i,
                                   i=i,
                                   metrics=BCL_metrics,
                                   embedding_save_path=embedding_save_path,
                                   loss_fn=loss_fn,
                                   model=model,
                                   RL_thresholds=RL_thresholds)
        # maintenance in window size and desert the last block
        # if i % args.window_size == 0 and i != data_split.shape[0] - 1:
        #     train_i = i
        #     model.load_state_dict(torch.load(args.resume_path + '/block_%d/models/best.pt' % i))

usage: ipykernel_launcher.py [-h] [--n_epochs N_EPOCHS] [--window_size WINDOW_SIZE] [--patience PATIENCE]
                             [--margin MARGIN] [--lr LR] [--batch_size BATCH_SIZE] [--hidden_dim HIDDEN_DIM]
                             [--out_dim OUT_DIM] [--heads HEADS] [--validation_percent VALIDATION_PERCENT]
                             [--use_hardest_neg] [--is_shared IS_SHARED] [--inter_opt INTER_OPT]
                             [--is_initial IS_INITIAL] [--sampler SAMPLER] [--cluster_type CLUSTER_TYPE]
                             [--threshold_start0 THRESHOLD_START0] [--RL_step0 RL_STEP0] [--RL_start0 RL_START0]
                             [--eps_start EPS_START] [--eps_step EPS_STEP] [--min_Pts_start MIN_PTS_START]
                             [--min_Pts_step MIN_PTS_STEP] [--use_cuda] [--data_path DATA_PATH]
                             [--mask_path MASK_PATH] [--resume_path RESUME_PATH] [--log_interval LOG_INTERVAL]
ipykernel_launcher.py: error: unrecognized argume

SystemExit: 2

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
